# Day 1: Introduction to RxRx19 dataset

Today we will be familiarizing ourselves with the dataset structure, and performing a few operations to understand it better.

## 1. Loading the datasets

The first step is to clone the repository with the data. The original dataset available on Kaggle has more than 305,520 images, with total size of >400 GB. Hence, we created a subset of 16,000 images and their labels.

> If you are curious to know how we created this subset, you can check [here](https://github.com/ai4all-sfu/comp-biology-2020/blob/master/day0-data-preprocessing.ipynb).



In [1]:
! git clone https://github.com/ai4all-sfu/comp-biology-2020.git

Cloning into 'comp-biology-2020'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 136 (delta 45), reused 69 (delta 19), pack-reused 32
Receiving objects: 100% (136/136), 244.77 MiB | 15.24 MiB/s, done.
Resolving deltas: 100% (52/52), done.


To check if the files are available, we use the code below. We should see two folders: 'sample_data' and 'comp-biology-2020'.

In [ ]:
!ls

Let us take a look at the structure of the dataset once again.

![alt text](https://drive.google.com/uc?id=1K8bj6FqO9mbz1hdbxK969B92PQvFxgdI)

RxRx19-images.zip consists of the image dataset. As discussed on the slides, two types of cells are being considered: HRCE and Vero cells. Each folder consists of cell images taken from 26 Plates. Each plate has ~26,000 images. Each cell site is passed through five channels, so each channel has ~5000 images. 

Let us take a look at the metadata and embeddings file. To reduce the size of the files, we saved them in pickle format. Let's unpickle them now.



In [2]:
import pandas as pd
import pickle

embeddings = pd.read_pickle('comp-biology-2020/embeddings.pkl', compression = 'xz')
metadata = pd.read_pickle('comp-biology-2020/metadata.pkl', compression = 'xz')

#changing the index
embeddings.set_index('site_id', inplace=True)

Let us print the head of metadata.pkl so that we can understand what it contains.

In [ ]:
metadata.head()

## 2. Understanding the Data

* 'site_id': it refers to the cell site under consideration. Every cell site has a unique site_id. As we discussed, every cell is analyzed under 4 sites, and each site is analyzed under 5 different channels. The format of site_id is as follows: 'experiment_plate_well_site'. So each of these will have an image each for the five different channels. Recall the comparison chart between them from the slides. That image is given below for your reference.

![alt text](https://drive.google.com/uc?id=1MhREWlcUghzmwiMMVz8GXU_p6zWZYonu)

* 'disease_condition': it refers to whether the cell is infected with the SARS-CoV-2 virus or not. In the original 'metadata.csv' there were three disease conditions: Active SARS-CoV-2 (the cell has been infected with the virus), UV Inactivated SARS-CoV-2, and Mock (mock preparations of SARS-CoV-2 on cells). We combined the disease conditions 'UV Inactivated SARS-CoV-2' and 'Mock' into one class: 'Inactive' (because they are similar), so that we can consider two distinct classes in our classification task: 'Active' and 'Inactive'.

* 'treatment': it refers to the drug that is used to treat the cell from the virus, 'treatment_conc' refers to the amount of concentration that the drug is used under. For an 'inactive' cell site, the value under 'treatment' and 'treatment_conc' will be NaN. 

Let us print embeddings.pkl and take a look at its head.

In [ ]:
embeddings.head()

In the above result, for every site_id we can observe 1024 feature values. These are lower-dimensional feature vectors (embeddings) for the image that provides some indication of what the image includes.

In our subset we have 16,000 images in total, chosen from all the four cell subfolders (HRCE-1, HRCE-2, Vero-2, and Vero-2). Each image is of dimensions 1024 x 1024 x 1. They are grayscale images. We will not be directly handling the images in our project. Instead, we will be using the embeddings.pkl file. 

Let us print the shape of metadata.pkl and embeddings.pkl.

In [ ]:
print("Metadata : ",metadata.shape)
print("Embeddings : ",embeddings.shape)

As we can see above, 'metadata' has 16,000 rows for the images and 10 columns for the metadata values for each image. 'embeddings' has 16,000 rows too with 1024 columns denoting 1024 feature values for each image.

Let us join 'metadata' and 'embeddings' to understand how they correlate better.

In [3]:
merged = pd.merge(embeddings, metadata, on=['site_id'], how='inner')
merged.head()

,site_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,...,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999,feature_1000,feature_1001,feature_1002,feature_1003,feature_1004,feature_1005,feature_1006,feature_1007,feature_1008,feature_1009,feature_1010,feature_1011,feature_1012,feature_1013,feature_1014,feature_1015,feature_1016,feature_1017,feature_1018,feature_1019,feature_1020,feature_1021,feature_1022,feature_1023,well_id,cell_type,experiment,plate,well,site,disease_condition,treatment,treatment_conc
0,HRCE-1_10_AA08_1,2.297975,0.022587,0.195894,0.014781,-1.100471,0.098462,0.242965,0.199119,-0.567304,-0.227210,-2.230506,-0.949541,-0.047310,0.533605,-0.290319,-1.508357,0.161617,0.380366,0.146494,-0.108530,-0.873874,0.056584,0.491351,0.080825,-1.321582,0.298009,0.190170,-0.864164,0.351424,0.536975,1.282918,-0.110490,-1.274246,0.943124,-1.541825,0.065526,-1.234762,0.295345,-1.824088,...,0.315934,-0.388910,-0.073659,0.390859,0.923492,0.642634,0.143817,0.619918,0.375651,0.347446,-0.236212,1.128898,0.012419,-1.073843,-0.257170,-0.714517,0.858910,0.012951,-0.503524,-1.609319,1.182596,0.586100,0.781158,-1.187773,-1.368313,-1.659620,-0.020060,-0.498513,0.349791,-0.286453,-1.424416,HRCE-1_10_AA08,HRCE,HRCE-1,10,AA08,1,inactive,NaN,NaN
1,HRCE-1_10_AA08_2,2.023117,0.055359,0.032669,-0.427921,-1.477027,0.555283,0.125149,0.024121,-0.310992,-0.035899,-2.317528,-0.571142,-0.190510,1.388795,-0.545442,-1.311690,0.067700,0.051449,0.281658,0.190968,-0.541562,0.122109,0.277549,-0.053468,-1.983142,0.453062,0.017778,-1.351487,0.600766,1.270639,1.074047,-0.062500,-1.360750,0.680048,-1.492564,0.104654,-1.461042,0.506609,-1.586523,...,0.370420,-0.908580,-0.119487,0.233618,0.823882,0.551085,-0.014573,1.159965,0.614649,0.434470,0.093707,1.066292,0.126574,-1.049656,-0.243583,-0.585675,0.663529,-0.127107,0.177727,-1.868272,1.141989,0.857226,0.687854,-1.667390,-1.059504,-1.597720,-0.335833,-0.643573,0.253038,0.145723,-1.794530,HRCE-1_10_AA08,HRCE,HRCE-1,10,AA08,2,inactive,NaN,NaN
2,HRCE-1_10_AA10_4,2.725714,0.097124,-0.251666,-0.245772,-0.611061,0.599592,0.098196,-0.359271,0.068162,-0.497672,-3.052315,0.072557,-0.220570,1.304541,-0.715440,-1.105000,-0.044301,-0.332337,0.173031,-0.191583,-0.540964,0.216181,0.587202,0.116111,-2.598186,0.853636,0.094363,-1.360084,0.333956,1.175048,0.694695,0.102681,-1.562893,0.513190,-1.809038,-0.241688,-1.753975,0.535202,-0.929908,...,-0.094190,-0.657681,-0.598234,0.280622,0.760488,0.412119,-0.066644,0.968393,0.613237,0.695272,0.209646,0.979274,0.368458,-0.925032,-0.176314,-0.899352,1.150092,0.490795,0.335611,-1.748789,1.015509,0.750727,0.885180,-1.421534,-1.284419,-1.417082,-0.065161,-0.390283,0.362758,0.033494,-1.130605,HRCE-1_10_AA10,HRCE,HRCE-1,10,AA10,4,active,Diaveridine,3.0
3,HRCE-1_10_AA16_1,2.525933,-0.161300,-0.485300,-0.169907,-0.828530,0.470630,0.684849,0.286412,-0.179563,-0.338007,-2.729603,-0.138102,-0.174999,0.560270,-0.890268,-1.483728,-0.084988,0.487842,0.466161,-0.412856,-0.633339,0.049574,0.873493,0.410658,-1.876541,0.806653,0.078848,-0.813504,0.353455,0.850163,0.900728,0.000268,-1.661532,0.586512,-1.927521,0.127292,-1.324757,0.346453,-1.238239,...,-0.065290,-0.360290,-0.072770,0.371843,0.597796,0.552014,0.200062,0.899955,0.576298,0.510481,-0.333821,1.069862,-0.011162,-1.220588,-0.333700,-0.563140,0.532201,-0.228118,-0.118126,-1.534507,0.994589,0.959356,0.722655,-1.339488,-1.445201,-1.371725,0.047971,-0.187304,0.134790,-0.127639,-1.530562,HRCE-1_10_AA16,HRCE,HRCE-1,10,AA16,1,inactive,NaN,NaN
4,HRCE-1_10_AA16_2,2.462103,-0.291629,-0.195823,-0.378053,-1.411975,0.364855,0.387182,0.188353,-0.026572,-0.068276,

Let us pick out the first row and take a clear look at the information that we get about each cell site.

In [ ]:
merged.iloc[0,:]

We will be using the merged dataframe in all of our exercises today. Let's eliminate all the other column values from 'merged' and just have our 'feature embeddings' for each image and the corresponding 'disease_condition'.

In [4]:
feat_disease = merged.iloc[:, list(range(1025)) + [-3]]
feat_disease

,site_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,...,feature_985,feature_986,feature_987,feature_988,feature_989,feature_990,feature_991,feature_992,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999,feature_1000,feature_1001,feature_1002,feature_1003,feature_1004,feature_1005,feature_1006,feature_1007,feature_1008,feature_1009,feature_1010,feature_1011,feature_1012,feature_1013,feature_1014,feature_1015,feature_1016,feature_1017,feature_1018,feature_1019,feature_1020,feature_1021,feature_1022,feature_1023,disease_condition
0,HRCE-1_10_AA08_1,2.297975,0.022587,0.195894,0.014781,-1.100471,0.098462,0.242965,0.199119,-0.567304,-0.227210,-2.230506,-0.949541,-0.047310,0.533605,-0.290319,-1.508357,0.161617,0.380366,0.146494,-0.108530,-0.873874,0.056584,0.491351,0.080825,-1.321582,0.298009,0.190170,-0.864164,0.351424,0.536975,1.282918,-0.110490,-1.274246,0.943124,-1.541825,0.065526,-1.234762,0.295345,-1.824088,...,-1.098585,0.753858,0.079735,0.000902,-0.457200,-0.152898,0.280296,-0.768077,0.315934,-0.388910,-0.073659,0.390859,0.923492,0.642634,0.143817,0.619918,0.375651,0.347446,-0.236212,1.128898,0.012419,-1.073843,-0.257170,-0.714517,0.858910,0.012951,-0.503524,-1.609319,1.182596,0.586100,0.781158,-1.187773,-1.368313,-1.659620,-0.020060,-0.498513,0.349791,-0.286453,-1.424416,inactive
1,HRCE-1_10_AA08_2,2.023117,0.055359,0.032669,-0.427921,-1.477027,0.555283,0.125149,0.024121,-0.310992,-0.035899,-2.317528,-0.571142,-0.190510,1.388795,-0.545442,-1.311690,0.067700,0.051449,0.281658,0.190968,-0.541562,0.122109,0.277549,-0.053468,-1.983142,0.453062,0.017778,-1.351487,0.600766,1.270639,1.074047,-0.062500,-1.360750,0.680048,-1.492564,0.104654,-1.461042,0.506609,-1.586523,...,-0.983013,0.355730,0.020508,0.567637,-0.209212,-0.159852,0.014123,-0.657308,0.370420,-0.908580,-0.119487,0.233618,0.823882,0.551085,-0.014573,1.159965,0.614649,0.434470,0.093707,1.066292,0.126574,-1.049656,-0.243583,-0.585675,0.663529,-0.127107,0.177727,-1.868272,1.141989,0.857226,0.687854,-1.667390,-1.059504,-1.597720,-0.335833,-0.643573,0.253038,0.145723,-1.794530,inactive
2,HRCE-1_10_AA10_4,2.725714,0.097124,-0.251666,-0.245772,-0.611061,0.599592,0.098196,-0.359271,0.068162,-0.497672,-3.052315,0.072557,-0.220570,1.304541,-0.715440,-1.105000,-0.044301,-0.332337,0.173031,-0.191583,-0.540964,0.216181,0.587202,0.116111,-2.598186,0.853636,0.094363,-1.360084,0.333956,1.175048,0.694695,0.102681,-1.562893,0.513190,-1.809038,-0.241688,-1.753975,0.535202,-0.929908,...,-0.640973,0.005899,0.005321,0.440640,-0.100642,-0.054449,0.023167,-0.835287,-0.094190,-0.657681,-0.598234,0.280622,0.760488,0.412119,-0.066644,0.968393,0.613237,0.695272,0.209646,0.979274,0.368458,-0.925032,-0.176314,-0.899352,1.150092,0.490795,0.335611,-1.748789,1.015509,0.750727,0.885180,-1.421534,-1.284419,-1.417082,-0.065161,-0.390283,0.362758,0.033494,-1.130605,active
3,HRCE-1_10_AA16_1,2.525933,-0.161300,-0.485300,-0.169907,-0.828530,0.470630,0.684849,0.286412,-0.179563,-0.338007,-2.729603,-0.138102,-0.174999,0.560270,-0.890268,-1.483728,-0.084988,0.487842,0.466161,-0.412856,-0.633339,0.049574,0.873493,0.410658,-1.876541,0.806653,0.078848,-0.813504,0.353455,0.850163,0.900728,0.000268,-1.661532,0.586512,-1.927521,0.127292,-1.324757,0.346453,-1.238239,...,-0.866113,0.363620,-0.183625,0.549890,-0.360297,0.254465,0.465468,-0.861761,-0.065290,-0.360290,-0.072770,0.371843,0.597796,0.552014,0.200062,0.899955,0.576298,0.510481,-0.333821,1.069862,-0.011162,-1.220588,-0.333700,-0.563140,0.532201,-0.228118,-0.118126,-1.534507,0.994589,0.959356,0.722655,-1.339488,-1.445201,-1.371725,0.047971,-0.187304,0.134790,-0

If you want to learn more about this dataset, explore the following links: 
* RxRx19: The First Morphological Imaging Dataset on SARS-CoV-2 Virus ([link](https://www.rxrx.ai/rxrx19) and [github](https://gist.github.com/bmabey/ae215f5c154cbc5c3b7e0a519e3d403b))
* RxRx19a COVID-19 Image Embeddings ([link](https://www.kaggle.com/tunguz/rxrx19a))


Now, we are going to understand the dataset by plotting graphs to analyze the relationships between different variables in the metadata and embeddings. Then, given an image, we are going to create feature embeddings using basic Computer Vision techniques.


On Day 2, we are going to reduce the dimension of our 'embeddings' dataset learn about several factor models, and how to scale the dataset.
And on day 3, we will be using 'embeddings' and parsing the values of  'disease_condition' for each of those embeddings and appending them to a labels list. We will train our model using this, and evaluate it against our test dataset. Our result will be a predicted 'disease_condition' label for a new cell image from the test dataset.

Let us plot the frequency count of disease condition 'active' in the 'feat_disease' dataframe. We use the in-built plot function in pandas for this.The function 'value_counts()' below is to count the number of occurences of each category.

In [ ]:
feat_disease['disease_condition'].value_counts().plot.bar()

### Activity 1

Plot the same graph, but in the form of a 'horizontal bar plot', and 'pie plot'. Try using the 'groupby' function from pandas to plot the graph. Add a legend and title too.


In [ ]:
#INSERT YOUR CODE HERE

### Activity 2

Plot the frequency distribution of all the 26 plates in the 'merged' dataframe. Use any two plot types of your choice. 
Try plotting the same, using plot functions from matplotlib.

In [ ]:
#INSERT YOUR CODE HERE


## Feature Extraction from an image

In our slides for Day 2, we had discussed two methods for feature extraction in an image. We will now be trying them out.

First, we use scikit-image which is a library containing a collection of algorithms for image processing. We will use the methods 'imread' and 'imshow' to read an image, and display it.

In [ ]:
import cv2
import numpy as np
from skimage.io import imread, imshow

image = imread('comp-biology-2020/supplement_images_day1/E08_s2_w1.png') 
print(image.shape)
imshow(image)



**1. Raw Pixel Feature Vector**

The simplest way is to extract the raw pixel feature vector. The image shape here is 1024 x 1024. Hence, the number of features should be 1,048,576. We can generate this using the reshape function from NumPy where we specify the dimension of the image:

In [ ]:
features = np.reshape(image, (1024*1024))

features.shape, features

((1048576,), array([2, 1, 1, ..., 1, 1, 1], dtype=uint8))

The shape of the feature vector is (1048576, ). This is nothing but (1024*1024, ).

Now, let us look at the second method for feature extraction.


**2. Extracting Edge Features**

On our slides, we had learnt that the idea behind edge detection is to extract edges as features and use that as the input for the model. Recall that an edge is a sharp change in color, or image intensity.

Let us again take a look at the example of a black square in a white background that we saw in the slides:

![alt text](https://drive.google.com/uc?id=1UfTCfQDgGfXZzj9rjVXSNBVY0upc--6D)

The 'vertical filter' in the above image is applied onto the red box by multiplying each pixel in the red box by each pixel in the filter element-wise. Each pixel in the result is achieved in exactly the same way. Then, we sum up the pixels in the result to get our vertical score. Here in our example, we get the vertical score/sum as -4. Thus, we know the pixel in question is part of a top vertical edge because we achieve the minimum value of -4.

In our example above, we have used the 'Sobel vertical filter'. In our coding exercises below, we will be using both: 'Sobel vertical filter', and 'Sobel horizontal filter'.




Let us first test it out on a sample image of a puppy. We read the image, and convert it to grayscale.

In [ ]:
puppy = imread('comp-biology-2020/supplement_images_day1/pupper.jpg', as_gray=True) 
puppy.shape, imshow(puppy)

Instead of doing the math behind applying a filter to an image, there is an in-built function in skimage library to get the result directly after applying the Sobel vertical and horizontal filters, and display it.

In [ ]:
from skimage.filters import sobel_h, sobel_v
from skimage import feature

#calculating horizontal edges using sobel kernel
edges_sobel_horizontal = sobel_h(puppy)
#calculating vertical edges using sobel kernel
edges_sobel_vertical = sobel_v(puppy)

imshow(edges_sobel_vertical, cmap='gray')

Display edges_sobel_horizontal as well and observe the differences between the two results.

Now, let us try to obtain a similar result by coding the math behind applying the filter to the image.

First read the image again without converting it to grayscale, Then, assign the Sobel filter values (these are fixed values) for vertical and horizontal filters.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

puppy = imread('comp-biology-2020/supplement_images_day1/pupper.jpg') 

#define the vertical filter
vertical_filter = [[-1,-2,-1], [0,0,0], [1,2,1]]

#define the horizontal filter
horizontal_filter = [[-1,0,1], [-2,0,2], [-1,0,1]]

#get the dimensions of the image
n,m,d = puppy.shape

### Activity 3 

Follow the following steps, and code using the method discussed in the slides.

Copy the original image into edges_img. We will be inserting our edge values into it one by one. Next, loop over the pixels in our original image: create a 3x3 box which is essentially a window/matrix considered in our original image, on which we will apply the filter. Then, multiply the values in the window with the ones in the vertical filter element-wise, and obtain the vertical score. Do the same for the horizontal filter. 

Now, in our example while studying how to apply a filter to an image, we only detected a single horizontal/vertical edge. In order to detect the horizontal edges, vertical edges, and edges that fall somewhere in between, we can combine the vertical and horizontal scores by calculating the Euclidean distance between them, and inserting this edge score into our image. This will give us the final result which we obtained in our previous example by directly using the in-built function.

In [ ]:
#initialize the edges image
edges_img = puppy.copy()

#loop over all pixels in the image
for row in range(3, n-2):
    for col in range(3, m-2):
        
        #create little local 3x3 box
        local_pixels = puppy[row-1:row+2, col-1:col+2, 0]
        
        #INSERT YOUR CODE HERE with the help of the comments given
        #apply the vertical filter
        
        #remap the vertical score
        
        #apply the horizontal filter
       
        #remap the horizontal score
        
        #combine the horizontal and vertical scores into a total edge score
        #edge_score = ...
        
        #insert this edge score into the edges image
        edges_img[row, col] = [edge_score]*3

Normalize the values in the image, and display.

In [ ]:
#remap the values in the 0-1 range in case they went out of bounds
edges_img = edges_img/edges_img.max()
imshow(edges_img)

Save your image results.

In [ ]:
from skimage.io import imsave
imsave('edge_puppy.jpg', edges_img) 

### Activity 4

1. Append the set of cell images given in the directory 'test_images_day1' to a list, and display them using subplot from matplotlib. Extract the raw pixel feature vector for these images. 

The *os* and *os.path* modules include functions to interact with the file system. We will be using some of them below to retrieve the image files from 'test_images_day1' directory.

In [ ]:
from os import listdir
from os.path import isfile, join

mypath='comp-biology-2020/test_images_day1'

#listdir will retrieve the list of files and directories in the current directory, isfile will check whether the specified path is an existing regular file or not
#join will join the path components 'mypath' and 'f'.
files = [f for f in listdir(mypath) if isfile(join(mypath,f))]
images = np.empty(len(files), dtype=object)
for n in range(0, len(files)):
  #INSERT YOUR CODE HERE for reading each image and appending to the images list.

#INSERT YOUR CODE HERE to display images using subplot.

In [ ]:
#Extract the raw pixel feature vectors for the list of test images
#INSERT YOUR CODE HERE


### Activity 5 (Advanced)
Use the edge detection method we just discussed, and extract the features for the same set of images from Activity 4. Use the inbuilt method for Sobel filter in skimage, and then the detailed math method. 

Hint: Use the same list 'images' that you used in the previous activity, and first display the results with the horizontal Sobel filter applied using subplot (simply reuse your code from the previous activity), and then display the results with the vertical Sobel filter applied.



In [ ]:
#INSERT YOUR CODE HERE
#initialize the edges image
edges_images = images.copy()
vertical_filter = [[-1,-2,-1], [0,0,0], [1,2,1]]
horizontal_filter = [[-1,0,1], [-2,0,2], [-1,0,1]]

#loop over all pixels in the images
for img in edges_images:
  n,m = img.shape
  for row in range(3, n-2):
      for col in range(3, m-2):
          local_pixels = img[row-1:row+2, col-1:col+2]
          vertical_transformed_pixels = vertical_filter*local_pixels
          vertical_score = vertical_transformed_pixels.sum()/4
          horizontal_transformed_pixels = horizontal_filter*local_pixels
          horizontal_score = horizontal_transformed_pixels.sum()/4      
          edge_score = (vertical_score**2 + horizontal_score**2)**.5
          
          img[row, col] = edge_score*3
  
#remap the values in the 0-1 range in case they went out of bounds
  img = img/img.max()

fig=plt.figure(figsize=(30, 30))
columns = 5
rows = 1
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(edges_images[i-1], cmap = 'gray')
fig.tight_layout(pad=3.0)    
plt.show()


### Activity 6 (Advanced)
Recall the explanation of Prewitt Filters, and Canny filters from the slides. Implement them using the inbuilt function in skimage, and we can discuss the results during the office hours. You can either use just the first image from test_images_day1, or all the images concatenated into a list (use 'images' from Activity 4)

Note: For each activity, save your image results using imsave, to display during the next session.

Feature extraction will not be used in the following days, this is just to give an introduction to image processing using basic Computer Vision techniques.

In [ ]:
#INSERT YOUR CODE HERE